In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from polarDensity_helper import Coord_Get, get_header_info
from dataclasses import dataclass

In [ ]:
@dataclass
class Site:
    inner_r: float
    outer_r: float
    thetabins: int
    title: str=""
    counts: list=None
    area: float=0
    densities: list=None
    Npeak: float=0
    mean: float=0


In [ ]:
def get_shell(some_data, site:Site, frames, Ntheta, dr):
    mask = np.logical_and(some_data[:,0] >= site.inner_r, some_data[:,1] <= site.outer_r)
    meta_shell = some_data[mask, 3:-1]
    nshells = (site.outer_r-site.inner_r)/dr
    assert nshells == int(nshells), "Error: non-integer number of shells"
    reshaped = np.reshape(meta_shell, (int(nshells),frames, Ntheta))
    flattened = np.sum(reshaped, axis=0)
    return flattened

def plot_density(site:Site, ax):
    dG = get_dg(site)

    ax.vlines(site.Npeak, 0, np.max(site.densities), color='k', linestyles='dashed', label="expected mean")
    ax.vlines(site.mean, 0, np.max(site.densities), color='red', linestyles='solid', label="actual mean")
    ax.plot(np.arange(len(site.densities)), site.densities, label=r"$\Delta G =$"+f"{np.round(dG,2)}kcal/mol")

    ax.legend()
    ax.set_xlabel(r"$N_\mathrm{beads}$")
    ax.set_ylabel("Probability Density")
    ax.set_title(site.title)

    return ax

def get_site_stats(site, exrho):
    new_site = site
    frequencies = np.bincount(site.counts.astype(int).flatten())
    new_site.densities = frequencies/np.sum(frequencies)
    
    new_site.Npeak = exrho*site.area
    new_site.mean = np.mean(site.counts)
    return new_site

def get_area(site:Site, dth):
    return np.mean([site.inner_r, site.outer_r])*(site.outer_r-site.inner_r)*dth*site.thetabins

def get_dg(site, RT= 0.6427483):
    p_lessthan = np.sum(site.densities[:int(site.Npeak)])
    p_greaterthan = np.sum(site.densities[int(site.Npeak):])
    dG = -RT*np.log(p_greaterthan/p_lessthan)
    return dG

In [ ]:
lipid = "POPG"
leaf = "upp"
data_root = Path("/Users/ezry/Projects/ELIC_PE_PG/trajectory_version/")
fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)


In [ ]:
def make_simple_site(inner_r, outer_r, nth, Ntheta, dr, dth, exrho, the_thetas, title):
    the_site = Site(inner_r, outer_r, nth, title)

    tmp_shell = get_shell(the_data, the_site, frames, Ntheta, dr)
    the_site.counts = tmp_shell[:,the_thetas]
    the_site.area = get_area(the_site, dth)
    the_site = get_site_stats(the_site, exrho)

    return the_site

In [ ]:
onebin = make_simple_site(25, 30, 1, Ntheta, dr, dth, exrho, [29], f"M4 site, one bin")
onebin_symmetric = make_simple_site(25, 30, 1, Ntheta, dr, dth, exrho, np.arange(5,30,6), "M4 site, one bin, symmetric sites")
fourbins = make_simple_site(25, 35, 2, Ntheta, dr, dth, exrho, np.arange(28,30,1), "M4 site, four bins")

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(15,15))

axes[0,0] = plot_density(onebin, axes[0,0])
axes[0,1] = plot_density(onebin_symmetric, axes[0,1])
axes[1,0] = plot_density(fourbins, axes[1,0])

plt.savefig(f"All_Site_Definitions.pdf")

In [ ]:
inner_r = 25
outer_r = 35
test_shell = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
the_thetas = np.arange(28,30,1)
site = np.sum(test_shell[:,the_thetas], axis=1)

ntheta = 2
site_area = get_area(inner_r, outer_r, dth, ntheta)
densities, Nexpected, experimental_mean = get_site_stats(site, site_area, exrho)
fig, ax = plot_density(densities, Nexpected, experimental_mean)
ax.set_title(f"Four bins: {lipid} {leaf}er leaflet between {inner_r} and {outer_r} Angstroms from origin")

plt.savefig(f"four_bins_{lipid}_{leaf}.pdf")

In [ ]:
inner_r = 25
outer_r = 35
large_shell = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
small_shell = get_shell(the_data, 20, 25, frames, Ntheta, dr)
site = np.sum(large_shell[:,28:30], axis=1)+small_shell[:,28]

ntheta = 2
site_area = get_area(inner_r, outer_r, dth, ntheta)
densities, Nexpected, experimental_mean = get_site_stats(site, site_area, exrho)
fig, ax = plot_density(densities, Nexpected, experimental_mean)
ax.set_title(f"Four bins: {lipid} {leaf}er leaflet between {inner_r} and {outer_r} Angstroms from origin, symmetric")

plt.savefig(f"four_bins_symmetric_{lipid}_{leaf}.pdf")

In [ ]:
#Part A the four contiguous bins
inner_r = 25
outer_r = 35
ntheta = 2
areaA = get_area(inner_r, outer_r, dth, ntheta)
large_shell = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
theta_A = np.sum([large_shell[:,4:30:6], small_shell[:,4:30:6]], axis=0)


#Part B, the one extra bin
inner_r = 20
outer_r = 25
ntheta = 1
areaB = get_area(inner_r, outer_r, dth, ntheta)
small_shell = get_shell(the_data, inner_r, outer_r, frames, Ntheta, dr)
theta_B = large_shell[:,5:30:6]


#aggregate:
site = theta_A+theta_B
site_area = areaA+areaB


densities, Nexpected, experimental_mean = get_site_stats(site, site_area, exrho)
fig, ax = plot_density(densities, Nexpected, experimental_mean)
ax.set_title(f"Five bins: {lipid} {leaf}er leaflet between {inner_r} and {outer_r} Angstroms from origin, symmetric")

plt.savefig(f"five_bins_symmetric_{lipid}_{leaf}.pdf")